## Q1. Running Elastic

In [2]:
%pip install elasticsearch==7.17.0


  Attempting uninstall: urllib3

    Found existing installation: urllib3 2.2.3

    Uninstalling urllib3-2.2.3:

      Successfully uninstalled urllib3-2.2.3

   ---------------------------------------- 0/2 [urllib3]
   ---------------------------------------- 0/2 [urllib3]
  Attempting uninstall: elasticsearch
   ---------------------------------------- 0/2 [urllib3]
    Found existing installation: elasticsearch 9.0.2
   ---------------------------------------- 0/2 [urllib3]
   -------------------- ------------------- 1/2 [elasticsearch]
    Uninstalling elasticsearch-9.0.2:
   -------------------- ------------------- 1/2 [elasticsearch]
      Successfully uninstalled elasticsearch-9.0.2
   -------------------- ------------------- 1/2 [elasticsearch]
   -------------------- ------------------- 1/2 [elasticsearch]
   -------------------- ------------------- 1/2 [elasticsearch]
   -------------------- ------------------- 1/2 [elasticsearch]
   -------------------- -------------------

In [3]:
!curl localhost:9200

{
  "name" : "0ab51432e96e",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "StwZ5YLCTRqKG10WIBPrsQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   539  100   539    0     0   2500      0 --:--:-- --:--:-- --:--:--  2495


### Getting the data

In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

In [7]:
from elasticsearch import Elasticsearch

In [8]:
from tqdm import tqdm

In [9]:
es_client = Elasticsearch(
    "http://localhost:9200",
)

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

C:\Users\jimoh\AppData\Local\Temp\ipykernel_19196\295183685.py:18: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es_client.indices.create(index=index_name, body=index_settings)


{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'course-questions'}

### Which function do you use for adding your data to elastic?
- index

In [11]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [01:08<00:00, 13.74it/s]


## Q3. Searching

In [12]:
query = 'How do execute a command on a Kubernetes pod?'

In [13]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)

C:\Users\jimoh\AppData\Local\Temp\ipykernel_19196\2928603148.py:15: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es_client.search(index=index_name, body=search_query)


In [14]:
result_scores = []
for hit in response['hits']['hits']:
    result_scores.append(hit['_score'])

In [16]:
result_scores

[44.50556, 35.433445, 33.70974, 33.2635, 32.589073]

### What's the score for the top ranking result?
- 44.50556

## Q4. Filtering

In [18]:
query = 'How do copy a file to a Docker container?'

In [22]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [19]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [20]:
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
  'section': '10. Kubernetes and TensorFlow Serving',
  'question': 'Kubernetes-dashboard',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t

In [21]:
for i, q in enumerate(result_docs, 1):
    print(f"Question {i}: {q['question']}")

Question 1: How do I debug a docker container?
Question 2: Kubernetes-dashboard
Question 3: How do I copy files from a different folder into docker container’s working directory?
Question 4: How to run a script while a web-server is working?
Question 5: How can I annotate a graph?


In [23]:
result_query = result_docs[2]['question']
result_query

'How do I copy files from a different folder into docker container’s working directory?'

### What's the 3rd question returned by the search engine?
- How do I copy files from a different folder into docker container’s working directory?

## Q5. Building a prompt

In [25]:
def search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [26]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    context = ""
    for doc in result_docs:
        context = context + context_template.format(question=doc['question'], text=doc['text']) + '\n\n'
    prompt = prompt_template.format(question=query, context=context)
    return prompt

In [27]:
query = 'How do copy a file to a Docker container?'
search_results = search(query)
prompt = build_prompt(query, search_results)

C:\Users\jimoh\AppData\Local\Temp\ipykernel_19196\4057711368.py:22: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es_client.search(index=index_name, body=search_query)


In [28]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: Kubernetes-dashboard
A: Deploy and Access the Kubernetes Dashboard
Luke

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:

In [29]:
len(prompt)

1919

### What's the length of the resulting prompt? (use the len function)
- 1919

## Q6. Tokens

In [30]:
!pip install tiktoken

   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ----------------------- ---------------- 524.3/894.9 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 894.9/894.9 kB 3.4 MB/s eta 0:00:00


In [31]:
import tiktoken

In [32]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [34]:
len(encoding.encode(prompt))

449

### How many tokens does our prompt have?
- 321